In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
import cv2
import os
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim

# 传统数据加载

In [ ]:

path_1 = 'E:\Download\high\\'
path_2 = 'E:\Download\low2\\'
fil_1 = os.listdir(path_1)
fil_2 = os.listdir(path_2)
#print(file_1)
#print(file_2)
train_data = []
for imagename in fil_1:
    image_path_1 =  path_1 + imagename
    image_path_2 =  path_2 + imagename
    high_image = cv2.imread(image_path_1)
    low_image = cv2.imread(image_path_2)
    '''
    cv2.namedWindow('high')
    cv2.namedWindow('low')
    cv2.imshow('high',high_image)
    cv2.imshow('low',low_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    '''
    HSI_H_image = cv2.cvtColor(high_image,cv2.COLOR_RGB2HSV)
    HSI_L_image = cv2.cvtColor(low_image,cv2.COLOR_RGB2HSV)
    H_H,H_S,H_I = cv2.split(HSI_H_image)
    #print(H_I)
    L_H,L_S,L_I = cv2.split(HSI_L_image)
    #print(L_I)
    H_I = H_I.reshape(1,1,400,600)
    L_I = L_I.reshape(1,1,400,600)
    #print(H_I)
    #print(L_I)
    #break
    train_data.append((torch.FloatTensor(H_I),torch.FloatTensor(L_I)))

In [ ]:
print(train_data[0][0].size())

# torchvision 数据加载

In [ ]:
from torchvision import transforms,datasets
from   PIL import Image
import torch
import torch.nn as nn
import os
import numpy as np
import cv2
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
import torch.nn.functional as F
class MyDatasets(Dataset):
    def __init__(self,text,transform=None):
        self.imgs = []
        self.labels = []
        file = open(text,'r')
        for i in file:
            img,label = i.split('\t')
            self.imgs.append(img)
            self.labels.append(label)
        self.transform = transform
        
    def __getitem__(self,index):
        image = cv2.imread(self.imgs[index])
        Label = cv2.imread(self.imgs[index])
        image = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
        Label = cv2.cvtColor(Label,cv2.COLOR_BGR2HSV)
        H_H,H_S,H_I = cv2.split(image)
        L_H,L_S,L_I = cv2.split(Label)
        img = H_I
        label = L_I
        if self.transform is not None:
            img = self.transform(img)
            label = self.transform(label)
        else:
            img = torch.FloatTensor(img.reshape(1,40,40))
            label = torch.FloatTensor(label.reshape(1,40,40))
        return img,label
    
    def __len__(self):
        return len(self.imgs)

In [ ]:
text = 'data_Retinex.txt'
batch_size = 64
MyDataset = MyDatasets(text)
train_data = DataLoader(MyDataset,batch_size=batch_size,shuffle = True,drop_last = True)

In [ ]:
class CovNet(nn.Module):
    def __init__(self):
        super(CovNet,self).__init__()
        self.module = nn.Sequential(
            nn.Conv2d(1,64,3,padding = 1,stride = 1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64,64,3,padding = 1,stride = 1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64,64,3,padding = 1,stride = 1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64,64,3,padding = 1,stride = 1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64,64,3,padding = 1,stride = 1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64,64,3,padding = 1,stride = 1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64,64,3,padding = 1,stride = 1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64,1,3,padding = 1,stride = 1),
            nn.ReLU()
            
        )
    def forward(self,data):
        return self.module(data)
#a = torch.randn(1,1,400,600)
#print(a)
#net = CovNet()
#print(net(a).shape)

In [ ]:
epochs = 15
device = torch.device('cuda:0')
ImgCovNet = CovNet().to(device)
loss_fun = nn.MSELoss().to(device)
optimizer = optim.Adam(ImgCovNet.parameters())

In [ ]:
loss_list = []
def Running():
    for epoch in range(epochs):
        lossvalue = 0
        for index,(datas,labels) in enumerate(train_data):
            data = datas.cuda()
            #print(data)
            label = labels.cuda()
            logits = ImgCovNet(data)
            loss = loss_fun(logits,label)
            #lossvalue+=loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_list.append(loss.item()/len(datas))
            print('epoch:{}\tbatch:{}\tave_loss:{}'.format(epoch+1,index+1,loss.item()/len(datas)))
        loss_list.append(lossvalue/len(train_data))
        print('epoch:{}\tave_loss:{}'.format(epoch+1,lossvalue/len(train_data)))

In [12]:
Running()

epoch:14	batch:492	ave_loss:0.3456580340862274
epoch:14	batch:493	ave_loss:0.23095697164535522
epoch:14	batch:494	ave_loss:0.055810198187828064
epoch:14	batch:495	ave_loss:0.06800452619791031
epoch:14	batch:496	ave_loss:0.21517330408096313
epoch:14	batch:497	ave_loss:0.13883888721466064
epoch:14	batch:498	ave_loss:1.0883755683898926
epoch:14	batch:499	ave_loss:0.14514611661434174
epoch:14	batch:500	ave_loss:0.23981693387031555
epoch:14	batch:501	ave_loss:0.04692505672574043
epoch:14	batch:502	ave_loss:0.4704170227050781
epoch:14	batch:503	ave_loss:0.11866890639066696
epoch:14	batch:504	ave_loss:1.0243899822235107
epoch:14	batch:505	ave_loss:0.15042346715927124
epoch:14	batch:506	ave_loss:0.30845707654953003
epoch:14	batch:507	ave_loss:0.21693596243858337
epoch:14	batch:508	ave_loss:0.057661060243844986
epoch:14	batch:509	ave_loss:0.21181349456310272
epoch:14	batch:510	ave_loss:0.07150197774171829
epoch:14	batch:511	ave_loss:0.07711934298276901
epoch:14	batch:512	ave_loss:0.788603842258

In [13]:
path = 'DCNN_2.pr'
torch.save(ImgCovNet.state_dict(),path)

In [37]:
loss =  []
for i  in range(15):
    loss.append(64*sum(loss_list[1137*i:(i+1)*1137])/1136)
#print(64*sum(loss_list[0:1137])/1136)
print(loss)
print(loss_list)

[2545.9635582910455, 83.87751080200705, 65.52513698708843, 56.02997477121756, 52.719446995728454, 44.61447672881711, 42.0567081255812, 34.7620602735961, 35.52898284750925, 31.635601364495887, 33.9364363665522, 33.81547784563941, 29.757302958255924, 28.767904965600497, 22.41902840546739]
[307.53411865234375, 285.0570068359375, 266.6393127441406, 249.09898376464844, 297.3218688964844, 313.79071044921875, 276.737548828125, 308.24383544921875, 263.7528381347656, 328.6197509765625, 261.2748718261719, 301.7210388183594, 272.9739990234375, 281.622314453125, 282.0572814941406, 306.4268798828125, 285.8444519042969, 267.6934509277344, 244.9717559814453, 284.1186218261719, 299.1197204589844, 298.06500244140625, 265.6100769042969, 273.4456481933594, 237.6776580810547, 290.1090087890625, 320.4395751953125, 267.1207275390625, 294.8591003417969, 267.1988525390625, 258.7030029296875, 309.6212463378906, 275.56329345703125, 260.19384765625, 277.91558837890625, 268.30023193359375, 302.9075012207031, 248.

In [ ]:
'''
def Running():
    for epoch in range(epochs):
        data_lst = []
        label_lst = []
        for index,(data,label) in enumerate(train_data):
           # print(data.shape)
           # print(label.shape)
            for i in range(10):
                for j in range(15):
                    data_lst.append(data[:,:,i*40:(i+1)*40,j*40:(j+1)*40])
                    label_lst.append(data[:,:,i*40:(i+1)*40,j*40:(j+1)*40])
                if (i+1)%10 == 0:
                    datas = torch.cat(data_lst,dim=0).cuda()
                    #print(datas.shape)
                    labels = torch.cat(label_lst,dim=0).cuda()
                    #print(labels.shape)
                    logits = ImgCovNet(datas)
                    loss = loss_fun(logits,labels)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    data_lst = []
                    label_lst = []
                    print('epoch:{}\t batch:{} \t loss:{}'.format(epoch,index+1,loss.item()))
             
            if (index+1)%2 == 0:
                data = torch.cat(data_lst,dim=0).cuda()
                label = torch.cat(label_lst,dim=0).cuda()
                logits = ImgCovNet(data)
                loss = loss_fun(logits,label)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                data_lst = []
                label_lst = []
                print('epoch:{}\t batch:{} \t loss:{}'.format(epoch,(index+1)/2,loss.item()))
'''

In [ ]:
Running()

In [ ]:
path = 'E:\Download\image\\123.jpg'
high_img = cv2.imread(path)
cv2.namedWindow('high_img')
cv2.namedWindow('low_img')
HSI = cv2.cvtColor(high_img,cv2.COLOR_RGB2HSV)
H,S,I = cv2.split(HSI)
I = I//6
print(I)
low_img = cv2.merge([H,S,I])
low_img = cv2.cvtColor(low_img,cv2.COLOR_HSV2BGR)
cv2.imwrite('E:\Download\image\\0113.jpg',low_img)
cv2.imshow('high_img',high_img)
cv2.imshow('low_img',low_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
path_1 = 'E:\Download\low2\\163.jpg'
low_image = cv2.imread(path_1)
low_image = cv2.cvtColor(low_image,cv2.COLOR_BGR2HSV)
H,S,I = cv2.split(low_image) 
#print(I)
image = torch.FloatTensor(I).reshape(1,1,400,600).to(device)
#print(image)
a = ImgCovNet(image)
#print(a)
b = a.reshape(400,600)
b  = b.cpu()
#print(b)
I = b.detach().numpy()
I = np.array(I,dtype = 'uint8')
re = cv2.merge([H,S,I])
re_img = cv2.cvtColor(re,cv2.COLOR_HSV2BGR)
low = cv2.imread(path_1)
#cv2.imwrite('E:\Download\test\low\\1.jpg',low)
#print('写入成功')
#cv2.imwrite('E:\Download\test\reconstruction\\1.jpg',re_img)
#print('写入成功')
cv2.namedWindow('low_image')

cv2.namedWindow('re_img')
cv2.imshow('re_img',re_img)
cv2.imshow('low_image',low)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import os
path_1 = r'E:\Download\test\low\\'

path_2 = r'E:\Download\test\reconstruction\\'

if not os.path.exists(path_1):
    os.makedirs(path_1)
if not os.path.exists(path_2):
    os.makedirs(path_2)
#image = cv2.imread(path_1)
cv2.imwrite(path_1+'6.jpg',low)
cv2.imwrite(path_2+'6.jpg',re_img)

In [ ]:
def ReConstruction(path_1,path_2):
    low_image = cv2.imread(path_1)
    low_image = cv2.cvtColor(low_image,cv2.COLOR_BGR2HSV)
    H,S,I = cv2.split(low_image) 
#print(I)
    image = torch.FloatTensor(I).reshape(1,1,400,600).to(device)
#print(image)
    a = ImgCovNet(image)
    print(a)
    b = a.reshape(400,600)
    b  = b.cpu()
    print(b)
    I = b.detach().numpy()
    I = np.array(I,dtype = 'uint8')
    re = cv2.merge([H,S,I])
    re_img = cv2.cvtColor(re,cv2.COLOR_HSV2BGR)
    low = cv2.imread(path_1)
    cv2.namedWindow('low_image')

    cv2.namedWindow('re_img')
    cv2.imshow('re_img',re_img)
    cv2.imshow('low_image',low)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
epoch:0	 batch:1 	 loss:27335.03125
epoch:0	 batch:2 	 loss:22525.244140625
epoch:0	 batch:3 	 loss:13668.892578125
epoch:0	 batch:4 	 loss:22249.322265625
epoch:0	 batch:5 	 loss:26448.2734375
epoch:0	 batch:6 	 loss:20327.662109375
epoch:0	 batch:7 	 loss:10969.900390625
epoch:0	 batch:8 	 loss:16134.89453125
epoch:0	 batch:9 	 loss:19878.60546875
epoch:0	 batch:10 	 loss:21279.732421875
epoch:0	 batch:11 	 loss:14770.19921875
epoch:0	 batch:12 	 loss:10368.7021484375
epoch:0	 batch:13 	 loss:22313.736328125
epoch:0	 batch:14 	 loss:15064.380859375
epoch:0	 batch:15 	 loss:13176.166015625
epoch:0	 batch:16 	 loss:24168.154296875
epoch:0	 batch:17 	 loss:14109.470703125
epoch:0	 batch:18 	 loss:11261.0263671875
epoch:0	 batch:19 	 loss:16713.95703125
epoch:0	 batch:20 	 loss:17433.986328125
epoch:0	 batch:21 	 loss:26284.181640625
epoch:0	 batch:22 	 loss:21625.357421875
epoch:0	 batch:23 	 loss:22805.759765625
epoch:0	 batch:24 	 loss:17554.7265625
epoch:0	 batch:25 	 loss:23321.2890625

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
with open('loss_curve_DNN.txt') as f:
    a =  f.readlines().strip()
    print(a)